<a href="https://colab.research.google.com/github/lauramoraes/Analise_de_Clusters/blob/master/5%20-%20Recomenda%C3%A7%C3%A3o%20de%20filmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recomendação de filmes

Retirado de: https://alyssaq.github.io/2015/20150426-simple-movie-recommender-using-svd/

In [0]:
import numpy as np
import pandas as pd

Carregando os dados já transformados para matriz

In [0]:
movie_data = pd.io.parsers.read_csv('https://www.dropbox.com/s/4fh04q6w4vn0kcv/movies.dat?dl=1',
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::', encoding="ISO-8859-1")

ratings_df = pd.read_csv("https://www.dropbox.com/s/ucwh8f86hk2hm6k/ml-1m.csv?dl=1", delimiter=",")
ratings_mat = ratings_df.values

Separando nome dos filmes do resto da matriz

In [0]:
movie_names = ratings_mat[:,0]
ratings_mat = ratings_mat[:,1:].astype(np.uint8)

Normalizando os dados

In [0]:
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T

Calculando SVD

In [0]:
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
U, S, V = np.linalg.svd(A)

Calculando similaridade de coseno

In [0]:
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 # Movie id starts from 1
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

# Helper function to print top N similar movies
def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.movie_id == movie_id].title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])

Vamos testar!

In [26]:
k = 50
movie_id = 1 # Grab an id from movies.dat
top_n = 10

sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, movie_id, top_n)
print_similar_movies(movie_data, movie_id, indexes)

Recommendations for Toy Story (1995): 

Toy Story (1995)
Men in Black (1997)
There's Something About Mary (1998)
Austin Powers: International Man of Mystery (1997)
Pleasantville (1998)
Clueless (1995)
Dunston Checks In (1996)
Full Monty, The (1997)
Wedding Singer, The (1998)
Grosse Pointe Blank (1997)
